In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [33]:

df_original = pd.read_csv("data/train_data.csv")
n_original = df_original.shape[0]
df_submit = pd.read_csv("data/sample_submission.csv")
df = pd.concat([df_original, df_submit], axis=0).reset_index(drop=True)

In [34]:

# CNN模型定义
class RegressionCNN(nn.Module):
    def __init__(self, sequence_length):
        super(RegressionCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32 * ((sequence_length // 2) - 1), 100)  # Adjust size accordingly
        self.regressor = nn.Linear(100, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.regressor(x)
        return x

# 特征构建函数（手工特征）
def siRNA_feat_builder(s: pd.Series, anti: bool = False):
    name = "anti" if anti else "sense"
    df = s.to_frame()
    df[f"feat_siRNA_{name}_seq_len"] = s.str.len()
    for pos in [0, -1]:
        for c in list("AUGC"):
            df[f"feat_siRNA_{name}_seq_{c}_{'front' if pos == 0 else 'back'}"] = (
                s.str[pos] == c
            )
    df[f"feat_siRNA_{name}_seq_pattern_1"] = s.str.startswith("AA") & s.str.endswith("UU")
    df[f"feat_siRNA_{name}_seq_pattern_2"] = s.str.startswith("GA") & s.str.endswith("UU")
    df[f"feat_siRNA_{name}_seq_pattern_3"] = s.str.startswith("CA") & s.str.endswith("UU")
    df[f"feat_siRNA_{name}_seq_pattern_4"] = s.str.startswith("UA") & s.str.endswith("UU")
    df[f"feat_siRNA_{name}_seq_pattern_5"] = s.str.startswith("UU") & s.str.endswith("AA")
    df[f"feat_siRNA_{name}_seq_pattern_6"] = s.str.startswith("UU") & s.str.endswith("GA")
    df[f"feat_siRNA_{name}_seq_pattern_7"] = s.str.startswith("UU") & s.str.endswith("CA")
    df[f"feat_siRNA_{name}_seq_pattern_8"] = s.str.startswith("UU") & s.str.endswith("UA")
    df[f"feat_siRNA_{name}_seq_pattern_9"] = s.str[1] == "A"
    df[f"feat_siRNA_{name}_seq_pattern_10"] = s.str[-2] == "A"
    df[f"feat_siRNA_{name}_seq_pattern_GC_frac"] = (
        s.str.contains("G") + s.str.contains("C")
    ) / s.str.len()
    return df.iloc[:, 1:]

# 数据准备及特征构建
df_publication_id = pd.get_dummies(df.publication_id)
df_publication_id.columns = [f"feat_publication_id_{c}" for c in df_publication_id.columns]
df_gene_target_symbol_name = pd.get_dummies(df.gene_target_symbol_name)
df_gene_target_symbol_name.columns = [f"feat_gene_target_symbol_name_{c}" for c in df_gene_target_symbol_name.columns]
df_gene_target_ncbi_id = pd.get_dummies(df.gene_target_ncbi_id)
df_gene_target_ncbi_id.columns = [f"feat_gene_target_ncbi_id_{c}" for c in df_gene_target_ncbi_id.columns]
df_gene_target_species = pd.get_dummies(df.gene_target_species)
df_gene_target_species.columns = [f"feat_gene_target_species_{c}" for c in df_gene_target_species.columns]
siRNA_duplex_id_values = df.siRNA_duplex_id.str.split("-|\.").str[1].astype("int")
siRNA_duplex_id_values = (siRNA_duplex_id_values - siRNA_duplex_id_values.min()) / (siRNA_duplex_id_values.max() - siRNA_duplex_id_values.min())
df_siRNA_duplex_id = pd.DataFrame(siRNA_duplex_id_values)
df_cell_line_donor = pd.get_dummies(df.cell_line_donor)
df_cell_line_donor.columns = [f"feat_cell_line_donor_{c}" for c in df_cell_line_donor.columns]
df_cell_line_donor["feat_cell_line_donor_hepatocytes"] = (df.cell_line_donor.str.contains("Hepatocytes")).fillna(False).astype("int")
df_cell_line_donor["feat_cell_line_donor_cells"] = df.cell_line_donor.str.contains("Cells").fillna(False).astype("int")
df_siRNA_concentration = df.siRNA_concentration.to_frame()
df_Transfection_method = pd.get_dummies(df.Transfection_method)
df_Transfection_method.columns = [f"feat_Transfection_method_{c}" for c in df_Transfection_method.columns]
df_Duration_after_transfection_h = pd.get_dummies(df.Duration_after_transfection_h)
df_Duration_after_transfection_h.columns = [f"feat_Duration_after_transfection_h_{c}" for c in df_Duration_after_transfection_h.columns]

# 合并所有特征
feats = pd.concat(
    [
        df_publication_id,
        df_gene_target_symbol_name,
        df_gene_target_ncbi_id,
        df_gene_target_species,
        df_siRNA_duplex_id,
        df_cell_line_donor,
        df_siRNA_concentration,
        df_Transfection_method,
        df_Duration_after_transfection_h,
        siRNA_feat_builder(df.siRNA_sense_seq, False),
        siRNA_feat_builder(df.siRNA_antisense_seq, True),
        df.iloc[:, -1].to_frame(),
    ],
    axis=1,
)

In [35]:

# 数据划分
X_train, X_test, y_train, y_test = train_test_split(
    feats.iloc[:n_original, :-1],
    feats.iloc[:n_original, -1],
    test_size=0.2,
    random_state=42,
)

# 假定每个序列的固定长度
sequence_length = 100

# 整数编码函数
def integer_encode_sequence(seq, sequence_length):
    mapping = {'A': 0, 'U': 1, 'G': 2, 'C': 3}
    # 截断序列
    seq = seq[:sequence_length]
    # 如果序列长度不足，填充到固定长度
    if len(seq) < sequence_length:
        seq += 'N' * (sequence_length - len(seq))
    return np.array([mapping.get(base, 4) for base in seq])  # 4 表示未知或填充的碱基 'N'

# 编码所有序列
X_seq = np.array([integer_encode_sequence(seq, sequence_length) for seq in df['siRNA_antisense_seq']])
X_seq = X_seq.reshape(X_seq.shape[0], 1, sequence_length)

# 按照与X_train相同的方式划分X_seq
X_seq_train, X_seq_test = train_test_split(X_seq, test_size=0.2, random_state=42)

# 确保X_seq_train和y_train的长度一致
assert X_seq_train.shape[0] == y_train.shape[0], "X_seq_train 和 y_train 的样本数量不匹配！"

# 创建数据集
train_dataset = TensorDataset(torch.tensor(X_seq_train, dtype=torch.float32), torch.tensor(y_train.values, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_seq_test, dtype=torch.float32), torch.tensor(y_test.values, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 定义CNN模型、优化器和损失函数
model = RegressionCNN(sequence_length)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

# 训练CNN模型
def train_model(model, train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1), target)
        loss.backward()
        optimizer.step()

# 提取CNN特征
def extract_features(model, loader):
    model.eval()
    features = []
    with torch.no_grad():
        for data, _ in loader:
            output = model(data)
            features.extend(output.view(-1).numpy())
    return np.array(features)

# CNN模型训练与特征提取
train_model(model, train_loader)
X_train_features = extract_features(model, train_loader)
X_test_features = extract_features(model, test_loader)

# 将CNN特征加入到原始特征中
X_train_combined = np.hstack([X_train, X_train_features.reshape(-1, 1)])
X_test_combined = np.hstack([X_test, X_test_features.reshape(-1, 1)])

# LightGBM模型训练与评估
train_data = lgb.Dataset(X_train_combined, label=y_train)
test_data = lgb.Dataset(X_test_combined, label=y_test, reference=train_data)


AssertionError: X_seq_train 和 y_train 的样本数量不匹配！

In [ ]:

def calculate_metrics(preds, data, threshold=30):
    y_pred = preds
    y_true = data.get_label()
    mae = np.mean(np.abs(y_true - y_pred))
    
    y_true_binary = ((y_true <= threshold) & (y_true >= 0)).astype(int)
    y_pred_binary = ((y_pred <= threshold) & (y_pred >= 0)).astype(int)

    mask = (y_pred >= 0) & (y_pred <= threshold)
    range_mae = mean_absolute_error(y_true[mask], y_pred[mask]) if np.sum(mask) > 0 else 100

    precision = (np.array(y_pred_binary) & y_true_binary).sum() / np.sum(y_pred_binary) if np.sum(y_pred_binary) > 0 else 0
    recall = (np.array(y_pred_binary) & y_true_binary).sum() / np.sum(y_true_binary) if np.sum(y_true_binary) > 0 else 0

    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)
    
    score = (1 - mae / 100) * 0.5 + (1 - range_mae / 100) * f1 * 0.5
    return "custom_score", score, True

def adaptive_learning_rate(decay_rate=0.8, patience=50):
    best_score = float("-inf")
    wait = 0

    def callback(env):
        nonlocal best_score, wait
        current_score = env.evaluation_result_list[-1][2]
        current_lr = env.model.params.get('learning_rate')

        if current_score > best_score:
            best_score = current_score
            wait = 0
        else:
            wait += 1

        if wait >= patience:
            new_lr = float(current_lr) * decay_rate
            wait = 0
            env.model.params['learning_rate'] = new_lr
            print(f"Learning rate adjusted to {env.model.params.get('learning_rate')}")

    return callback

def train(feats, n_original):
    n_splits = 10
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    gbms = []
    for fold, (train_idx, val_idx) in enumerate(kf.split(feats.iloc[:n_original, :]), 1):
        print(f"Starting fold {fold}")
        X_train, X_val = feats.iloc[train_idx, :-1], feats.iloc[val_idx, :-1]
        y_train, y_val = feats.iloc[train_idx, -1], feats.iloc[val_idx, -1]

        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

        params = {
            "boosting_type": "gbdt",
            "objective": "regression",
            "metric": "None",
            "max_depth": 8,
            "num_leaves": 63,
            "min_data_in_leaf": 2,
            "learning_rate": 0.05,
            "feature_fraction": 0.9,
            "lambda_l1": 0.1,
            "lambda_l2": 0.2,
            "verbose": -1,
            "num_threads": 8,
        }

        adaptive_lr = adaptive_learning_rate(decay_rate=0.9, patience=1000)
        gbm = lgb.train(
            params,
            train_data,
            num_boost_round=25000,
            valid_sets=[val_data],
            feval=calculate_metrics,
            callbacks=[
                adaptive_lr,
                lgb.log_evaluation(period=200, show_stdv=True),
                lgb.early_stopping(stopping_rounds=int(25000*0.1), first_metric_only=True, verbose=True, min_delta=0.00001)
            ]
        )
        valid_score = gbm.best_score["valid_0"]["custom_score"]
        print(f"Fold {fold} best valid score: {valid_score}")
        gbms.append(gbm)

    return gbms

# 训练最终模型
trained_gbms = train(feats, n_original)

In [ ]:
# 预测并保存结果
y_pred = np.mean([gbm.predict(feats.iloc[n_original:, :-1]) for gbm in trained_gbms], axis=0)

In [ ]:

df_submit["mRNA_remaining_pct"] = y_pred
df_submit.to_csv("submission.csv", index=False)